### Implementation of the codes developed, openTURNS style


In [1]:
import openturns as ot
import numpy as np
import PureBeamExample as pbe #Toy model to analyse
import KarhunenLoeveFieldSensitivity as klfs #Openturns integration of the analysis algorithm

In this notebook, we will show how the codes developed for the sensitivity anakysis on models governed by stochastic fields can be easily integrated in the actual openturns environment. 

We will use the toy example of the bending beam to demonstrate the method.

##### The behaviour of the algorithms is highly inspired of the ones found in the openTURNS API

###### The codes are composed of four parts that behave together :
    ot.AggregatedKarhunenLoeveResults
    ot.KarhunenLoeveSobolIndicesExperiment
    ot.KarhunenLoeveGeneralizedFunctionWrapper
    ot.SobolKarhunenLoeveFieldSensitivityAlgorithm
    

##### We are in the case where the model to analyse takes as an input mulitple stochastic fields and random variables. The model also returns multiple fields and scalar values. 

To be able to use random variables in our AggregatedKarhunenLoeveObject, we express our random variables as a constant stochastic field. In this way, we can still use our KarhunenLoeveDecomposition on it, but we will have to convert the constant field into a scalar value again just before passing it to our model. 
This must be coded manually. 

### Definition of the input processes and random variables :

In [2]:
# Function to convert a scalar distribution into a field : 

def variablesAsProcess(distribution, mesh):
    '''Function to transform a scalar distribution into 
    a constant process defined over a mesh
    '''
    basis = ot.Basis([ot.SymbolicFunction(['x'],['1'])])
    lawAsprocess = ot.FunctionalBasisProcess(distribution, basis, mesh)
    lawAsprocess.setName(distribution.getName())
    return lawAsprocess

#############################################################

# First step : DEFINING THE PROCESSES AND RANDOM VARIABLES
# First Process E_ : Youngs modulus
# dimension 
dimension = 1
#grid
#Number of elements:
NElem = [100]
mesher = ot.IntervalMesher(NElem)
lowBound = [0] #mm
highBound = [1000] #mm
interval = ot.Interval(lowBound,highBound)
mesh = mesher.build(interval)

#Covariance model Young's modulus
amplitude0 = [50000]*dimension
scale0 = [300]*dimension
nu0 = 13/3
model0 = ot.MaternModel(scale0, amplitude0, nu0)
# Karhunen Loeve decomposition of process 
algorithm = ot.KarhunenLoeveP1Algorithm(mesh, model0, 1e-3)
algorithm.run()
resultsE = algorithm.getResult()
resultsE.setName('E_')

# Second Process D_ : Diameter
amplitude = [.3]*dimension
scale = [250]*dimension
nu = 13/3
model1 = ot.MaternModel(scale, amplitude, nu)
algorithm = ot.KarhunenLoeveP1Algorithm(mesh, model1, 1e-3)
algorithm.run()
resultsD = algorithm.getResult()
resultsD.setName('D_')

#############################################################

# SECOND STEP : Definition of the random variables
# random variable for the density of the material (kg/m³)
sigma       = 750
nameD       = 'Rho'
RV_Rho = ot.Normal(0, sigma)
RV_Rho.setName(nameD)
# random variable for the position of the force   (mm) 
sigma_f      = 50
namePos     = 'FP'
RV_Fpos = ot.Normal(0, sigma_f)
RV_Fpos.setName(namePos)
# random variable for the norm of the force    (N)
sigma_Fnor    = 5.5
nameNor       = 'FN'
RV_Fnorm  = ot.Normal(0, sigma_Fnor)
RV_Fnorm.setName(nameNor)

#############################################################

# Then convert the distributions t processes over a mesh
SP_Rho = variablesAsProcess(RV_Rho, mesh)
SP_Fpos = variablesAsProcess(RV_Fpos, mesh)
SP_Fnorm = variablesAsProcess(RV_Fnorm, mesh)

#############################################################

# Then we can do the Karhunen Loeve decomposition of the distributions : 
algorithm0 = ot.KarhunenLoeveP1Algorithm(mesh, SP_Rho.getCovarianceModel(), 1e-3)
algorithm0.run()
resultsRho = algorithm0.getResult()
resultsRho.setName('Rho_')

algorithm1 = ot.KarhunenLoeveP1Algorithm(mesh, SP_Fpos.getCovarianceModel(), 1e-3)
algorithm1.run()
resultsFpos = algorithm1.getResult()
resultsFpos.setName('Fpos_')

algorithm2 = ot.KarhunenLoeveP1Algorithm(mesh, SP_Fnorm.getCovarianceModel(), 1e-3)
algorithm2.run()
resultsFnor = algorithm2.getResult()
resultsFnor.setName('Fnorm_')

# Finally we get a list of aggregated KarhunenLoeve results
listOfKLRes = [resultsE, resultsD, resultsRho, resultsFpos, resultsFnor]


### NOTE :

When defining a distribution as a process, as shown above, it is of utmost importance to modifiy his function so that it takes as an input a constant process, not a distribution anymore. This can also be easily avoided, if you take the first value of the field generated.

###### Always multiple ways to proceed :

In the example presented here, 2 ways of approaching the problem are possible. First we can create all our processes individually, decompose them individually with Karhunen Loeve, and pass a list of KarhunenLoeveResults to our new function wrapper. 
But this is only necessary if we would have meshes of different shapes, or if we would have defined the scalar distributions on smaller meshes. But as all processes are defined over the same mesh here, we could have aggregated the Processes, and directly decompose the AggregatedProcess. 

###### Note2 :

When using the **KarhunenLoeveP1Algorithm** function, only the covariance function is decomposed this means **all data 'bout the means and the trend functions are FORGOTTEN**. You have to add these constants **Directly in yout function!** 


##### First tests with new architecture. 
our base is a list of Karhunen Loeve Results. This will be passed into a contructor called AggregatedKarhunenLoeveResults. This  class is necessary as it is used to pass between the one dimensional space of the KarhunenLoeve Coefficients to the arbitrary dimension of the field linked to it. 
This class keeps all the data about the decomposition and allows to work on lists of processes

In [3]:
AggregatedKLObj = klfs.AggregatedKarhunenLoeveResults(listOfKLRes)

In [4]:
AggregatedKLObj

class = AggregatedKarhunenLoeveResults| name = Unnamed| Aggregation Order = 5| Threshold = 0.001| Covariance Model 0 = class=MaternModel scale=class=Point name=Unnamed dimension=1 values=[300] amplitude=class=Point name=Unnamed dimension=1 values=[50000] nu=4.33333| Covariance Model 1 = class=MaternModel scale=class=Point name=Unnamed dimension=1 values=[250] amplitude=class=Point name=Unnamed dimension=1 values=[0.3] nu=4.33333| Covariance Model 2 = class=RankMCovarianceModel, variance=class=Point name=Unnamed dimension=1 values=[562500], covariance=class=CovarianceMatrix dimension=0 implementation=class=MatrixImplementation name=Unnamed rows=0 columns=0 values=[], basis=class=Basis coll=[class=Function name=Unnamed implementation=class=FunctionImplementation name=Unnamed description=[x,y0] evaluationImplementation=class=SymbolicEvaluation name=Unnamed inputVariablesNames=[x] outputVariablesNames=[y0] formulas=[1] gradientImplementation=class=SymbolicGradient name=Unnamed evaluation=c

### Now we initialize our toy example : 

In [5]:
## First: beam initialization with mesh
PureBeam = pbe.PureBeam(mesh)

### Then we pass our model into our function wrapper. We have two functions : 
- One for single evaluations
- One for batch evaluations
##### - Only one of the functions is required. 
#### In the wrapper we also pas the AggregatedKarhunenLoeveResults object, as well as the number of outputs the function has. 
## WARNING : 
#### It is not the dimension of the output (eg. the dimension of a field) but it represents the numer of different outputs the function has, as it is assumed that the function returns a tuple of scalars, fields etc. 

In [6]:
# Now that we have our function, we can pass it in our wrapper! 
FUNC = klfs.KarhunenLoeveGeneralizedFunctionWrapper(
                                AggregatedKarhunenLoeveResults = AggregatedKLObj,
                                func        = PureBeam.singleEval, 
                                func_sample = PureBeam.batchEval,
                                n_outputs   = 2)


#### Once the model is created as the AggregatedKarhunenLoeveResult object, we can begin our DOE (Design Of Experiment)
- For this, we use our class called **KarhunenLoeveSobolIndicesExperiment**. 
- This last class aims to be similar to the class ot.SobolIndicesExperiment. 
##### Generation of the input design : 

In [7]:
N = 20
SobolExperiment = klfs.KarhunenLoeveSobolIndicesExperiment(AggregatedKLObj, N ,False)
inputDesign = SobolExperiment.generate()

Samples A and B of size 20 and dimension 18
Experiment of size 140 and dimension 18


##### Evaluation of the function on the input design

In [8]:
outputDesign = FUNC(inputDesign)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:    9.6s


shape deflection:  (140, 103)  should be [N,10X] something
deflection std deviation  6.239814057942748
Using the batch evaluation function. Assumes that the outputs are in the 
same order than for the single evaluation function. This one should only 
return ProcessSamples, Samples, Lists or numpy arrays.
Element is iterable, assumes that first dimension is size of sample
Shape is (140, 102) and dtype is <class 'numpy.float64'>
Element 0 of the output tuple returns process samples of dimension 1
Element is iterable, assumes that first dimension is size of sample
Shape is (140,) and dtype is <class 'numpy.float64'>
Element 1 of the output tuple returns samples of dimension 1


[Parallel(n_jobs=-1)]: Done 140 out of 140 | elapsed:   10.8s finished
/home/simady/anaconda/envs/stochastic_field_env/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [9]:
FieldSensitivityAnalysis = klfs.SobolKarhunenLoeveFieldSensitivityAlgorithm()
FieldSensitivityAnalysis.setDesign(inputDesign, outputDesign, N)
FieldSensitivityAnalysis.setEstimator(ot.SaltelliSensitivityAlgorithm())

if you also pass the data to compute it
Otherwise, the behavior will be unreliable
There are 5 indices to get for 2 outputs with dimensions 102, 1 each.
SobolIndicesName ['Rho_', 'E_', 'Fnorm_', 'D_', 'Fpos_']


In [10]:
FieldSensitivityAnalysis.getFirstOrderIndices()

[[class=Field name=Sobol_Y_0_Rho_ description=[v0,v0] implementation=class=FieldImplementation name=Sobol_Y_0_Rho_ mesh=class=Mesh name=1D_Grid dimension=1 vertices=class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=102 dimension=1 data=[[0],[0.00990099],[0.019802],[0.029703],[0.039604],[0.049505],[0.0594059],[0.0693069],[0.0792079],[0.0891089],[0.0990099],[0.108911],[0.118812],[0.128713],[0.138614],[0.148515],[0.158416],[0.168317],[0.178218],[0.188119],[0.19802],[0.207921],[0.217822],[0.227723],[0.237624],[0.247525],[0.257426],[0.267327],[0.277228],[0.287129],[0.29703],[0.306931],[0.316832],[0.326733],[0.336634],[0.346535],[0.356436],[0.366337],[0.376238],[0.386139],[0.39604],[0.405941],[0.415842],[0.425743],[0.435644],[0.445545],[0.455446],[0.465347],[0.475248],[0.485149],[0.49505],[0.50495],[0.514851],[0.524752],[0.534653],[0.544554],[0.554455],[0.564356],[0.574257],[0.584158],[0.594059],[0.60396],[0.613861],[0.623762],[0.633663],[0.643564],[0.6534

In [11]:
FieldSensitivityAnalysis.getFirstOrderIndicesInterval()

[class=Interval name=Bounds_Sobol_Y_0 dimension=5 lower bound=class=Point name=Unnamed dimension=5 values=[0.999102,0.483259,0.999102,0.138541,0.608806] upper bound=class=Point name=Unnamed dimension=5 values=[1.01317,0.9188,1.01317,0.831901,1.07824] finite lower bound=[1,1,1,1,1] finite upper bound=[1,1,1,1,1],
 class=Interval name=Bounds_Sobol_Y_1 dimension=5 lower bound=class=Point name=Unnamed dimension=5 values=[-0.456658,0.674648,0.997656,0.968967,0.77058] upper bound=class=Point name=Unnamed dimension=5 values=[0.370376,1.12448,1.02511,1.15674,1.11233] finite lower bound=[1,1,1,1,1] finite upper bound=[1,1,1,1,1]]

In [12]:
FieldSensitivityAnalysis.getAggregatedFirstOrderIndices()

[class=PointWithDescription name=Sobol_Y_0 dimension=5 description=[Y_0_Rho_,Y_0_E_,Y_0_Fnorm_,Y_0_D_,Y_0_Fpos_] values=[1.00004,0.731932,1.00004,0.533964,0.832405],
 class=PointWithDescription name=Sobol_Y_1 dimension=5 description=[Y_1_Rho_,Y_1_E_,Y_1_Fnorm_,Y_1_D_,Y_1_Fpos_] values=[0.0782087,0.94105,1.00017,1.05824,0.934373]]

In [13]:
FieldSensitivityAnalysis.getAggregatedTotalOrderIndices()

[class=PointWithDescription name=Sobol_Y_0 dimension=5 description=[Y_0_Rho_,Y_0_E_,Y_0_Fnorm_,Y_0_D_,Y_0_Fpos_] values=[-3.61294e-05,0.268068,-3.6187e-05,0.466036,0.167595],
 class=PointWithDescription name=Sobol_Y_1 dimension=5 description=[Y_1_Rho_,Y_1_E_,Y_1_Fnorm_,Y_1_D_,Y_1_Fpos_] values=[0.921791,0.0589501,-0.00017141,-0.058239,0.0656273]]

In [14]:
FieldSensitivityAnalysis.getFirstOrderIndicesDistribution()

Cannot convert distribution to field, dimensional correlation lost.
Cannot convert distribution to field, dimensional correlation lost.


[class=KernelMixture name=Y_0 kernel=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[0] sigma=class=Point name=Unnamed dimension=1 values=[1] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1] bandwidth=class=Point name=Unnamed dimension=5 values=[0.000966956,0.0571566,0.000966948,0.0767147,0.061616] sample=class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=100 dimension=5 data=[[1.00097,0.788192,1.00097,0.485245,0.764348],[1.00317,0.707582,1.00317,0.427359,0.958901],[1.00186,0.854249,1.00186,0.418543,0.787679],[1.00201,0.645932,1.00201,0.419613,0.904836],[1.00266,0.768553,1.00266,0.757849,0.839468],[1.0005,0.596435,1.0005,0.66303,0.783344],[1.00495,0.602146,1.00495,0.289869,1.04658],[1.011,0.806177,1.011,0.559406,0.627701],[1.00251,0.775898,1.00251,0.537508,0.752213],[1.00456,0.771822,1.00456,0.398132,1.01496],[1.00184,0.668123

In [15]:
FieldSensitivityAnalysis.getTotalOrderIndices()

[[class=Field name=TotalOrderSobol_Y_0_Rho_ description=[v0,v0] implementation=class=FieldImplementation name=TotalOrderSobol_Y_0_Rho_ mesh=class=Mesh name=1D_Grid dimension=1 vertices=class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=102 dimension=1 data=[[0],[0.00990099],[0.019802],[0.029703],[0.039604],[0.049505],[0.0594059],[0.0693069],[0.0792079],[0.0891089],[0.0990099],[0.108911],[0.118812],[0.128713],[0.138614],[0.148515],[0.158416],[0.168317],[0.178218],[0.188119],[0.19802],[0.207921],[0.217822],[0.227723],[0.237624],[0.247525],[0.257426],[0.267327],[0.277228],[0.287129],[0.29703],[0.306931],[0.316832],[0.326733],[0.336634],[0.346535],[0.356436],[0.366337],[0.376238],[0.386139],[0.39604],[0.405941],[0.415842],[0.425743],[0.435644],[0.445545],[0.455446],[0.465347],[0.475248],[0.485149],[0.49505],[0.50495],[0.514851],[0.524752],[0.534653],[0.544554],[0.554455],[0.564356],[0.574257],[0.584158],[0.594059],[0.60396],[0.613861],[0.623762],[0.633663

In [16]:
FieldSensitivityAnalysis.getTotalOrderIndicesDistribution()

Cannot convert distribution to field, dimensional correlation lost.
Cannot convert distribution to field, dimensional correlation lost.


[class=KernelMixture name=Y_0 kernel=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[0] sigma=class=Point name=Unnamed dimension=1 values=[1] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1] bandwidth=class=Point name=Unnamed dimension=5 values=[0.19416,0.184206,0.19416,0.236373,0.138734] sample=class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=100 dimension=5 data=[[-0.0518828,0.160896,-0.0518828,0.463843,0.18474],[0.17505,0.470637,0.17505,0.75086,0.219319],[0.0405406,0.188152,0.0405407,0.623858,0.254722],[0.406095,0.762173,0.406095,0.988491,0.503268],[0.00717357,0.241282,0.00717358,0.251986,0.170368],[-0.145514,0.25855,-0.145514,0.191955,0.0716409],[0.403894,0.8067,0.403894,1.11898,0.362269],[-0.233213,-0.0283907,-0.233213,0.218381,0.150086],[-0.0301546,0.196455,-0.0301546,0.434844,0.220139],[0.172232,0.404974,0.172232,0.77

In [17]:
FieldSensitivityAnalysis.getTotalOrderIndicesInterval()

[class=Interval name=BoundsTotalOrderSobol_Y_0 dimension=5 lower bound=class=Point name=Unnamed dimension=5 values=[-0.493426,-0.223714,-0.493426,-0.164628,-0.179925] upper bound=class=Point name=Unnamed dimension=5 values=[0.880401,1.27216,0.880401,1.59187,0.927169] finite lower bound=[1,1,1,1,1] finite upper bound=[1,1,1,1,1],
 class=Interval name=BoundsTotalOrderSobol_Y_1 dimension=5 lower bound=class=Point name=Unnamed dimension=5 values=[0.349651,-0.449424,-0.457313,-0.572196,-0.397147] upper bound=class=Point name=Unnamed dimension=5 values=[2.18014,1.16671,1.00366,1.00861,1.01359] finite lower bound=[1,1,1,1,1] finite upper bound=[1,1,1,1,1]]

In [18]:
FieldSensitivityAnalysis.getSecondOrderIndices()

The second order indices flag is not set to true.
Have you passed the right sample to make this calculus?
